In [1]:
import pandas as pd
import numpy as np

In [13]:
# read in merged csv
merge = pd.read_csv('merged_files.csv')
merge.head()

,Unnamed: 0,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,...,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed
0,0,211028,2005,2005-06-13,The Rose Bowl,1,0.1,England,Australia,ME Trescothick,...,B Lee,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,211028,2005,2005-06-13,The Rose Bowl,1,0.2,England,Australia,ME Trescothick,...,B Lee,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,211028,2005,2005-06-13,The Rose Bowl,1,0.3,England,Australia,GO Jones,...,B Lee,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,211028,2005,2005-06-13,The Rose Bowl,1,0.4,England,Australia,GO Jones,...,B Lee,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,211028,2005,2005-06-13,The Rose Bowl,1,0.5,England,Australia,GO Jones,...,B Lee,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#merge.columns
merge['batting_team'].unique()

array(['England', 'Australia', 'New Zealand', 'South Africa', 'Pakistan',
       'Sri Lanka', 'West Indies', 'India', 'Kenya', 'Scotland',
       'Zimbabwe', 'Bangladesh', 'Bermuda', 'Netherlands', 'Ireland',
       'Afghanistan', 'Canada', 'Nepal', 'Hong Kong',
       'United Arab Emirates', 'Papua New Guinea', 'Oman', 'ICC World XI',
       'Philippines', 'Vanuatu', 'United States of America', 'Germany',
       'Italy', 'Ghana', 'Namibia', 'Uganda', 'Botswana', 'Nigeria',
       'Guernsey', 'Denmark', 'Norway', 'Jersey', 'Thailand', 'Malaysia',
       'Maldives', 'Singapore', 'Qatar', 'Kuwait', 'Cayman Islands',
       'Portugal', 'Spain', 'Gibraltar', 'Bhutan', 'Saudi Arabia',
       'Bahrain', 'Iran', 'Belgium', 'Luxembourg', 'Czech Republic',
       'Isle of Man', 'Bulgaria', 'Romania', 'Austria', 'Greece',
       'Serbia', 'Malta', 'France', 'Sweden', 'Rwanda', 'Finland',
       'Hungary', 'Estonia', 'Cyprus', 'Switzerland', 'Seychelles',
       'Malawi', 'Lesotho', 'Swaziland', 

In [15]:
# read in list of players 
players = pd.read_csv('players.csv')
players['runs/ball'] = pd.Series(dtype='float')
players.head()

,name,position,country,runs/ball


In [16]:
# index through to get only rows that match text file
# here, we get only plays from teams participating in the 2024 World Cup

with open('teams.txt') as file:
    teams_list = file.read().splitlines()
df = merge.loc[(merge['bowling_team'].isin(teams_list)) | (merge['batting_team'].isin(teams_list))]
print(df, df.shape)

        Unnamed: 0  match_id   season  start_date                    venue  \
0                0    211028     2005  2005-06-13            The Rose Bowl   
1                1    211028     2005  2005-06-13            The Rose Bowl   
2                2    211028     2005  2005-06-13            The Rose Bowl   
3                3    211028     2005  2005-06-13            The Rose Bowl   
4                4    211028     2005  2005-06-13            The Rose Bowl   
...            ...       ...      ...         ...                      ...   
477264         231   1412551  2023/24  2024-01-02  Sharjah Cricket Stadium   
477265         232   1412551  2023/24  2024-01-02  Sharjah Cricket Stadium   
477266         233   1412551  2023/24  2024-01-02  Sharjah Cricket Stadium   
477267         234   1412551  2023/24  2024-01-02  Sharjah Cricket Stadium   
477268         235   1412551  2023/24  2024-01-02  Sharjah Cricket Stadium   

        innings  ball batting_team          bowling_team       

In [ ]:
# we will use only data from 2020 onward (last 4 years)
df[['start-year', 'start-day', 'start-month']] = df['start_date'].str.split('-', expand=True)
df = df.loc[df['start-year'].astype(int) >= 2020]
df = df.drop(['start-year', 'start-day', 'start-month'], axis=1)
df.head()

## Calculate Stats

In [18]:
# TODO: loop through df to calculate runs/ball for all bowlers and hitters/strikers
# Here, looking at games from all countries
_df = (merge
       .loc[:, ['striker', 'runs_off_bat', 'extras']]
       .groupby(['striker'], as_index = False)
       .sum())
_df_sorted = _df.sort_values(by='runs_off_bat', ascending=False)

In [19]:
num_bowls = (df
       .loc[:, ['striker', 'bowling_team', 'batting_team']]
       .groupby(['striker', 'batting_team'], as_index = False)
       .count()
       .rename(columns = {'bowling_team' : 'n_bowls'}))

df2 = _df.merge(num_bowls, on = ['striker'])

In [20]:
df2['runs_per_bowl'] = (df2['runs_off_bat'] + df2['extras']) / df2['n_bowls']
df2['weighted_runs_per_bowl'] = df2['runs_per_bowl'] * df2['n_bowls']
df2.sort_values(by = 'weighted_runs_per_bowl', ascending = False)

,striker,runs_off_bat,extras,batting_team,n_bowls,runs_per_bowl,weighted_runs_per_bowl
1144,V Kohli,4043,166,India,1050,4.008571,4209.0
891,RG Sharma,4013,172,India,966,4.332298,4185.0
185,Babar Azam,3698,137,Pakistan,1808,2.121128,3835.0
666,MJ Guptill,3545,186,New Zealand,811,4.600493,3731.0
841,PR Stirling,3357,139,Ireland,1172,2.982935,3496.0
...,...,...,...,...,...,...,...
1086,T Sumiya,0,0,Mongolia,1,0.000000,0.0
618,LU Igalagamage,0,0,Sri Lanka,3,0.000000,0.0
625,M Altankhuyag,0,0,Mongolia,9,0.000000,0.0
840,PP Shaw,0,0,India,1,0.000000,0.0


In [21]:
us = df2[df2['batting_team'] == 'United States of America']
us.sort_values(by = 'weighted_runs_per_bowl', ascending = False)

,striker,runs_off_bat,extras,batting_team,n_bowls,runs_per_bowl,weighted_runs_per_bowl
998,SR Taylor,315,20,United States of America,199,1.683417,335.0
1178,XM Marshall,261,31,United States of America,72,4.055556,292.0
83,Aaron Jones,275,14,United States of America,187,1.545455,289.0
659,MD Patel,267,22,United States of America,133,2.172932,289.0
525,JS Malhotra,248,11,United States of America,112,2.312500,259.0
361,G Singh,163,23,United States of America,134,1.388060,186.0
771,NK Patel,100,7,United States of America,83,1.289157,107.0
979,SJ Modani,93,5,United States of America,71,1.380282,98.0
679,MO Kain,71,2,United States of America,50,1.460000,73.0
541,K Gore,47,10,United States of America,27,2.111111,57.0
